In [1]:
import os
import sys
import glob
import argparse
import matplotlib.pyplot as plt

from keras import __version__
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, RMSprop

Using TensorFlow backend.


https://deeplearningsandbox.com/how-to-use-transfer-learning-and-fine-tuning-in-keras-and-tensorflow-to-build-an-image-recognition-94b0b02444f2

In [2]:
IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
FC_SIZE = 1024
NB_IV3_LAYERS_TO_FREEZE = 172 # unfreeze the top 2 inception blocks

def setup_to_transfer_learn(model, base_model):
    """Freeze all layers and compile the model"""
    for layer in base_model.layers:
        layer.trainable = False
    
    for layer in model.layers:
        print(layer.name, ': ', layer.trainable)
    model.compile(optimizer=RMSprop(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])


def add_new_last_layer(base_model, nb_classes):
    """Add last layer to the convnet
      Args:
        base_model: keras model excluding top
        nb_classes: # of classes
      Returns:
        new keras model with last layer
    """
    
    x = base_model.output

    # GlobalAveragePooling2D converts the MxNxC tensor output into a 1xC tensor where C is the # of channels.
    x = GlobalAveragePooling2D()(x)
    x = Dense(FC_SIZE, activation='relu')(x) # new FC layer, random init
    # softmax function on the output to squeeze the values between [0,1]
    predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer

    model = Model(inputs=base_model.input, outputs=predictions)
    return model


def setup_to_finetune(model):
    """Freeze the bottom NB_IV3_LAYERS and retrain the remaining top layers.
  note: NB_IV3_LAYERS corresponds to the top 2 inception blocks in the inceptionv3 arch
  Args:
    model: keras model
  """
    for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
        layer.trainable = False
    for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
        layer.trainable = True
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])


def train():
    """Use transfer learning and fine-tuning to train a network on a new dataset"""
    
    nb_train_samples = 5000
    nb_classes = 10
    nb_val_samples = 1000
    nb_epoch = 10
    batch_size = 4 #16
    train_dir = 'data/train'
    val_dir = 'data/validation'
    output_model_file = 'inceptionv3_clothing_classifier.h5'

    # data prep
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input, # zero-centers our image data
        # rescale = 1./255,
#           rotation_range=30,
#           width_shift_range=0.2,
#           height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    
    val_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input
    )
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )
    
    validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=batch_size,
        class_mode='categorical'
    )

    # setup model
    # leave out the weights of the last fully connected layer
    base_model = InceptionV3(weights='imagenet', include_top=False) #include_top=False excludes final FC layer
    model = add_new_last_layer(base_model, nb_classes)
    
    # transfer learning
    setup_to_transfer_learn(model, base_model)

    history_tl = model.fit_generator(
        train_generator,
        epochs=nb_epoch,
        steps_per_epoch=nb_train_samples//batch_size,
        validation_data=validation_generator,
        validation_steps=nb_val_samples//batch_size
        #,
        #class_weight='auto'
    )
    
    print('Begin Fine Tuning')
    # fine-tuning
    setup_to_finetune(model)
    
    history_ft = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples//batch_size,
        epochs=nb_epoch,
        validation_data=validation_generator,
        validation_steps=nb_val_samples//batch_size
        #,
        #class_weight='auto'
    )
    
    model_json = model.to_json()
    with open("incep_filter_clothing_classifier.json", 'w') as json_file:
        json_file.write(model_json)
    
    model.save(output_model_file)


In [3]:
train()

Found 5000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
input_1 :  False
conv2d_1 :  False
batch_normalization_1 :  False
activation_1 :  False
conv2d_2 :  False
batch_normalization_2 :  False
activation_2 :  False
conv2d_3 :  False
batch_normalization_3 :  False
activation_3 :  False
max_pooling2d_1 :  False
conv2d_4 :  False
batch_normalization_4 :  False
activation_4 :  False
conv2d_5 :  False
batch_normalization_5 :  False
activation_5 :  False
max_pooling2d_2 :  False
conv2d_9 :  False
batch_normalization_9 :  False
activation_9 :  False
conv2d_7 :  False
conv2d_10 :  False
batch_normalization_7 :  False
batch_normalization_10 :  False
activation_7 :  False
activation_10 :  False
average_pooling2d_1 :  False
conv2d_6 :  False
conv2d_8 :  False
conv2d_11 :  False
conv2d_12 :  False
batch_normalization_6 :  False
batch_normalization_8 :  False
batch_normalization_11 :  False
batch_normalization_12 :  False
activation_6 :  False
activation_8 :  False
act

1250/1250 [==============================] - 227s - loss: 1.5166 - acc: 0.5226 - val_loss: 1.3922 - val_acc: 0.5810
Epoch 5/10
1250/1250 [==============================] - 227s - loss: 1.4319 - acc: 0.5448 - val_loss: 1.3024 - val_acc: 0.5930
Epoch 6/10
1250/1250 [==============================] - 227s - loss: 1.3918 - acc: 0.5478 - val_loss: 1.2548 - val_acc: 0.6200
Epoch 7/10
1250/1250 [==============================] - 227s - loss: 1.3484 - acc: 0.5604 - val_loss: 1.2256 - val_acc: 0.5990
Epoch 8/10
1250/1250 [==============================] - 227s - loss: 1.3098 - acc: 0.5662 - val_loss: 1.1574 - val_acc: 0.6340
Epoch 9/10
1250/1250 [==============================] - 227s - loss: 1.2717 - acc: 0.5818 - val_loss: 1.1817 - val_acc: 0.6160
Epoch 10/10
1250/1250 [==============================] - 227s - loss: 1.2468 - acc: 0.5868 - val_loss: 1.1115 - val_acc: 0.6310
Epoch 1/10
1250/1250 [==============================] - 366s - loss: 1.1151 - acc: 0.6278 - val_loss: 0.8498 - val_acc: 0

Epoch 1/10
1250/1250 [==============================] - 230s - loss: 2.0870 - acc: 0.3006 - val_loss: 1.8965 - val_acc: 0.4250
Epoch 2/10
1250/1250 [==============================] - 227s - loss: 1.8024 - acc: 0.4486 - val_loss: 1.6371 - val_acc: 0.5150
Epoch 3/10
1250/1250 [==============================] - 227s - loss: 1.6245 - acc: 0.4930 - val_loss: 1.4899 - val_acc: 0.5640
Epoch 4/10
1250/1250 [==============================] - 227s - loss: 1.5166 - acc: 0.5226 - val_loss: 1.3922 - val_acc: 0.5810
Epoch 5/10
1250/1250 [==============================] - 227s - loss: 1.4319 - acc: 0.5448 - val_loss: 1.3024 - val_acc: 0.5930
Epoch 6/10
1250/1250 [==============================] - 227s - loss: 1.3918 - acc: 0.5478 - val_loss: 1.2548 - val_acc: 0.6200
Epoch 7/10
1250/1250 [==============================] - 227s - loss: 1.3484 - acc: 0.5604 - val_loss: 1.2256 - val_acc: 0.5990
Epoch 8/10
1250/1250 [==============================] - 227s - loss: 1.3098 - acc: 0.5662 - val_loss: 1.1574 - val_acc: 0.6340
Epoch 9/10
1250/1250 [==============================] - 227s - loss: 1.2717 - acc: 0.5818 - val_loss: 1.1817 - val_acc: 0.6160
Epoch 10/10
1250/1250 [==============================] - 227s - loss: 1.2468 - acc: 0.5868 - val_loss: 1.1115 - val_acc: 0.6310
Epoch 1/10
1250/1250 [==============================] - 366s - loss: 1.1151 - acc: 0.6278 - val_loss: 0.8498 - val_acc: 0.7120
Epoch 2/10
1250/1250 [==============================] - 364s - loss: 0.9272 - acc: 0.6896 - val_loss: 0.8035 - val_acc: 0.7500
Epoch 3/10
1250/1250 [==============================] - 364s - loss: 0.7979 - acc: 0.7414 - val_loss: 0.7634 - val_acc: 0.7500
Epoch 4/10
1250/1250 [==============================] - 364s - loss: 0.7279 - acc: 0.7602 - val_loss: 0.6825 - val_acc: 0.7620
Epoch 5/10
1250/1250 [==============================] - 364s - loss: 0.6840 - acc: 0.7772 - val_loss: 0.6948 - val_acc: 0.7790
Epoch 6/10
1250/1250 [==============================] - 364s - loss: 0.6060 - acc: 0.8024 - val_loss: 0.6900 - val_acc: 0.7830
Epoch 7/10
1250/1250 [==============================] - 364s - loss: 0.5571 - acc: 0.8192 - val_loss: 0.6952 - val_acc: 0.7840
Epoch 8/10
1250/1250 [==============================] - 364s - loss: 0.5145 - acc: 0.8306 - val_loss: 0.7060 - val_acc: 0.7850
Epoch 9/10
1250/1250 [==============================] - 364s - loss: 0.4856 - acc: 0.8454 - val_loss: 0.6554 - val_acc: 0.8100
Epoch 10/10
1250/1250 [==============================] - 364s - loss: 0.4489 - acc: 0.8502 - val_loss: 0.7246 - val_acc: 0.7920